In [1]:
#Setting Spark with MinIO
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType
from pyspark.sql.functions import *
import os
from pyspark.sql import SparkSession
S3_ACCESS_KEY = os.getenv("S3_ACCESS_KEY")
S3_SECRET_KEY = os.getenv("S3_SECRET_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")
BUCKET_NAME = os.getenv("BUCKET_NAME")
POSTGRES_USER=os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD=os.getenv("POSTGRES_PASSWORD")
POSTGRES_ENDPOINT=os.getenv("POSTGRES_ENDPOINT")

spark = SparkSession.builder \
    .appName("MinIOReader") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.jars", "/home/harshithts/jars/postgresql-42.7.3.jar") \
    .config("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")\
    .getOrCreate()


spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")



25/11/13 15:15:42 WARN Utils: Your hostname, harshithts-HP-Pavilion-Gaming-Laptop-15-ec2xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.2 instead (on interface wlo1)
25/11/13 15:15:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/harshithts/Documents/sturctruing/env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/harshithts/.ivy2/cache
The jars for the packages stored in: /home/harshithts/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3c0212ca-9eec-4b5d-a340-d9906086497c;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 340ms :: artifacts dl 11ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	------------------------

In [2]:
account_df = spark.read.format("parquet")\
    .load(f"s3a://{BUCKET_NAME}/silver/processed-accounts/*.parquet")
profile_df = spark.read.format("parquet")\
    .load(f"s3a://{BUCKET_NAME}/silver/processed-profiles/*.parquet")
merchant_df = spark.read.format("parquet")\
    .load(f"s3a://{BUCKET_NAME}/silver/processed-merchants/*.parquet")



25/11/13 15:15:51 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [3]:


profile_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/data") \
    .option("dbtable", "public.USERS") \
    .option("user", POSTGRES_USER) \
    .option("password", POSTGRES_PASSWORD) \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()
account_df.write \
    .format("jdbc") \
    .option("url", POSTGRES_ENDPOINT) \
    .option("dbtable", "public.ACCOUNTS") \
    .option("user", POSTGRES_USER) \
    .option("password", POSTGRES_PASSWORD) \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()
merchant_df.write \
    .format("jdbc") \
    .option("url", POSTGRES_ENDPOINT) \
    .option("dbtable", "public.MERCHANTS") \
    .option("user", POSTGRES_USER) \
    .option("password", POSTGRES_PASSWORD) \
    .option("driver", "org.postgresql.Driver") \
    .mode("append") \
    .save()